In [18]:
# from google.colab import drive
# drive.mount('/content/drive')

In [19]:
# %cd /content/drive/MyDrive/GRAD/Thesis/MAPR

In [20]:
# !ls

In [21]:
!python --version

Python 3.9.21


In [22]:
# !pip show transformers

In [23]:
#Load the LLM and the MEGAVUL's train dataframe
import json
import pandas as pd
import gc

# Input and output file paths
input_file = r"C:\Users\Administrator\Downloads\VulScribeR-20250404T140824Z-001\VulScribeR\code\megavul_simple.json"
# vul_output_file = r"/content/drive/MyDrive/GRAD/Thesis/MAPR/megavul_vul.json"
# clean_output_file = r"/content/drive/MyDrive/GRAD/Thesis/MAPR/megavul_clean.json"

# Set the chunk size (adjust as needed)
chunksize = 10000

# Create empty lists to store data for each category
vul_data = []
clean_data = []

# Read the JSON file in chunks
# for chunk in pd.read_json(input_file, lines=True, chunksize=chunksize):
#     # Separate data based on 'is_vul'
#     print(chunk)
#     vul_chunk = chunk[chunk['is_vul'] == True]
#     clean_chunk = chunk[chunk['is_vul'] == False]

#     # Append data to the respective lists
#     vul_data.extend(vul_chunk.to_dict('records'))
#     clean_data.extend(clean_chunk.to_dict('records'))
#     gc.collect()  # Run garbage collection
with open(input_file, "r", encoding="utf-8") as f:
    json_data = json.load(f)

mega_vul_df = pd.DataFrame(json_data)


In [24]:

vul_df = mega_vul_df[mega_vul_df['is_vul'] == True]
clean_df = mega_vul_df[mega_vul_df['is_vul'] == False]

In [25]:
# # !wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh
# !wget https://repo.anaconda.com/miniconda/Miniconda3-py39_23.9.0-0-Linux-x86_64.sh
# !chmod +x Miniconda3-py39_23.9.0-0-Linux-x86_64.sh
# !./Miniconda3-py39_23.9.0-0-Linux-x86_64.sh -b -f -p /usr/local
# !conda update conda -y

In [26]:
import torch

In [27]:

# !conda create --name vulscriber-cluster-env python=3.8 -y
# !conda activate vulscriber-cluster-env
# !conda install pytorch==1.10.0 torchvision==0.11.1 torchaudio==0.10.0 cudatoolkit=11.3 -c pytorch -y
# !pip install transformers>4.37.0 pandas scikit-learn accelerate>=0.26.0

In [28]:

# !conda create --name vulscriber-cluster-env1 python=3.9 -y
# !conda activate vulscriber-cluster-env1
# !conda install pytorch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1  pytorch-cuda=11.8 -c pytorch -c nvidia -y
# !conda install transformers>4.37.0 pandas scikit-learn -y
# !conda install conda-forge::accelerate -y


In [29]:
# !conda install conda-forge::accelerate -y

In [30]:
# !pip install accelerate>=0.26.0

In [31]:
import torch
print(torch.cuda.is_available())  # Should print: True
print(torch.cuda.device_count())  # Should print: 1
print(torch.cuda.get_device_name(0))  # Should print: NVIDIA GeForce RTX 3060
print(torch.cuda.current_device())  # Should print 0


True
1
NVIDIA GeForce RTX 3060
0


In [32]:
import json
from typing import List

# Define the classes to match the JSON structure


class ScoredCodeSnippet:
    def __init__(self, id: str, body: str, header: str, score: float, cluster: int):
        self.id = id
        self.body = body
        self.header = header
        self.score = score
        self.cluster = cluster

    def __repr__(self) -> str:
        return self.__str__()

    def __str__(self):
        return (f"ScoredCodeSnippet(id='{self.id}', "
                # Print only the first 60 characters of the body for brevity
                f"body='{self.body[:60]}...', "
                f"header='{self.header}', "
                f"cluster='{self.cluster}', "
                f"score={self.score})")


class SearchItem:
    def __init__(self, searchItemId: str, scoredCodeSnippets: List[ScoredCodeSnippet]):
        self.searchItemId = searchItemId
        self.scoredCodeSnippets = scoredCodeSnippets
        self.best, self.max_score, self.avg_score = self.__find_max(scoredCodeSnippets)

    def __find_max(self, snippets: List[ScoredCodeSnippet]):
        avg = 0
        max_score = -1
        best_match = None
        for snippet in snippets:
            avg += snippet.score /len(snippets)
            if snippet.score > max_score:
                max_score = snippet.score
                best_match = snippet.id
        return best_match, max_score, avg

    def get_flattened(self):
        items = []
        for snippet in self.scoredCodeSnippets:
            newItem = SearchItem(self.searchItemId, [snippet])
            items.append(newItem)
        return items

    def __repr__(self) -> str:
        return self.__str__()

    def __str__(self):
        snippets_str = "\n  ".join(str(snippet)
                                   for snippet in self.scoredCodeSnippets)
        return (f"SearchItem(searchItemId='{self.searchItemId}', "
                f"scoredCodeSnippets=[\n  {snippets_str}\n])")


def flatten_search_items(search_results: List[SearchItem]):
    items = []
    for result in search_results:
        for item in result.get_flattened():
            items.append(item)
    return items

def json_to_classes(json_data):
    search_items = []
    for item in json_data:
        searchItemId = item["searchItemId"]
        scoredCodeSnippets = []
        for snippet in item["scoredCodeSnippets"]:
            cluster = -1
            if "clusterIndex" in snippet:
                cluster = int(snippet["clusterIndex"])
            scoredCodeSnippets.append(
                ScoredCodeSnippet(
                    id=snippet["id"],
                    body=snippet["body"],
                    header=snippet["header"],
                    score=float(snippet["score"]),
                    cluster=cluster
                )
            )
        search_items.append(SearchItem(searchItemId, scoredCodeSnippets))
    return search_items


def read_rag(file_name):
    result = None
    with open(f"../code/RAG/{file_name}.json", "r") as file:
        data = json.load(file)
        result = json_to_classes(data)

    return result

def read_naive_code2code_ext_rag():
    return read_rag("results_full_code2code_ext")

def read_naive_code2code_rag():
    return read_rag("results_full_code2code")

def read_naive_code2code_clustered_rag():
    return read_rag("results_full_code2code_clustered")

def read_random():
    return read_rag("results_random")

def read_random_fair():
    return read_rag("results_random_fair")

def read_naive_full_header_rag():
    return read_rag("results_full_header")


def read_naive_full_bigvul_header_rag():
    return read_rag("results_full_header_bigvul")

def read_naive_full_func_rag():
    return read_rag("results_full_func")

In [33]:
# import openai
import re
import concurrent.futures
import threading
import time
# from rag import SearchItem, ScoredCodeSnippet
from typing import List, Callable
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"  # Explicitly select GPU 0
CODEQWEN_ARCHITECTURE = "Qwen/Qwen2.5-Coder-1.5B"
# CODEQWEN_ARCHITECTURE = "Qwen/CodeQwen1.5-7B"
place_holders = {1: "$MY_UNIQUE_PLACE_HOLDER_1", 2: "$MY_UNIQUE_PLACE_HOLDER_2",
                 3: "$MY_UNIQUE_PLACE_HOLDER_3", 4: "$MY_UNIQUE_PLACE_HOLDER_4", 5: "$MY_UNIQUE_PLACE_HOLDER_5"}

templates = {"base": """
Here are two code snippets specified below, modify code snippet No.2 in a way that it includes the logic of code No.1:
Code Snippet 1: "$MY_UNIQUE_PLACE_HOLDER_1"

Code Snippet 2: "$MY_UNIQUE_PLACE_HOLDER_2"

Put the generated code inside ```C ```
""",
             "extra": """
Here are two code snippets specified below, modify code snippet No.2 in a way that it includes the logic of code No.1:
Code Snippet 1: "$MY_UNIQUE_PLACE_HOLDER_1"

Code Snippet 2: "$MY_UNIQUE_PLACE_HOLDER_2"

Note that the following lines from the first code snippet have a high priority to be in the final modified code:

Lines separated by /~/ : "$MY_UNIQUE_PLACE_HOLDER_3"

Put the generated code inside ```C ```. (Do not include comments)
""", "with_example_vardef": """Here are three code snippets (each containing a function) specified below, look how code snippet AA is changed into BB, introduce these changes to code snippet CC.

Code Snippet AA: "$MY_UNIQUE_PLACE_HOLDER_1"

Code Snippet BB: "$MY_UNIQUE_PLACE_HOLDER_2"

Code Snippet CC: "$MY_UNIQUE_PLACE_HOLDER_3"

Note that the following lines from code snippet CC have a high priority to be in the final modified code:

Lines separated by /~/ : "$MY_UNIQUE_PLACE_HOLDER_4"

Put the generated code inside ```C ```”, and note that the output should be single function with a similar name and parameters as the code snippet CC, and the return statements should be placed after the logic of the code. $MY_UNIQUE_PLACE_HOLDER_5
""",
"vardef": "(Don't Forget to add these variables to function's input arguments: $MY_UNIQUE_PLACE_HOLDER_1)",

"without_example_vardef": """Here are two code snippets specified below, modify code snippet BB in a way that it includes the logic of code AA:

Code Snippet AA: "$MY_UNIQUE_PLACE_HOLDER_1"

Code Snippet BB: "$MY_UNIQUE_PLACE_HOLDER_2"

Note that the following lines from the code snippet AA have a high priority to be in the final modified code:

Lines separated by /~/ : "$MY_UNIQUE_PLACE_HOLDER_3"

Put the generated code inside ```C ``` and note that the output should be single function with a similar function definition as code snippet BB, and the return statements should be placed after the logic of the code. $MY_UNIQUE_PLACE_HOLDER_4""",
"ext":"""Here's two code snippets AA and BB, each including a function. Add some parts of the logic of AA to BB in a way that the result would include all of BB but is not equal to BB. You can add variables to BB to produce a more correct code.

Code Snippet AA: "$MY_UNIQUE_PLACE_HOLDER_1"

Code Snippet BB: "$MY_UNIQUE_PLACE_HOLDER_2"

Put the generated code inside ```C ``` and note that the final result should a function that takes all the input args of BB and more if required. (Do not include comments)
""", "ext_wl":
"""Here's two code snippets AA and BB, each including a function. Add some parts of the logic of AA to BB in a way that the result would include important lines of BB with some parts from AA. You can add variables to BB to produce a more correct code.

Code Snippet AA: "$MY_UNIQUE_PLACE_HOLDER_1"

Code Snippet BB: "$MY_UNIQUE_PLACE_HOLDER_2"

The following lines from BB are important and should be included in the final result, the rest may be changed or even removed if they have no relation to these lines: (Lines are separated via /~/)

"$MY_UNIQUE_PLACE_HOLDER_3"

Put the generated code inside ```C ``` and note that the final result should a function that takes all the input args of BB and more if required. (Do not include comments)
""", "mutation_naive": """Here's a code snippets including a function. Except for the important lines mentioned below and the function's name and input variables, mutate the rest of the code so it is different from the original while keeping the original semantics.

Code Snippet: "$MY_UNIQUE_PLACE_HOLDER_1"

The following lines are important and should be included in the final result, the rest may be changed or even removed if they have no relation to these lines: (Lines are separated via /~/)

"$MY_UNIQUE_PLACE_HOLDER_2"

Put the generated code inside ```C ```. (Do not include comments)
""", "mutation": """Here's a code snippet including a function. Except for the important lines mentioned below, mutate the rest of the code snippet so it is different from the original while keeping the original semantics.
To do so you can use one or more of the following rules:

Rules: 1-> Replace the local variables’ identifiers with new non-repeated identifiers
2-> Replace the for statement with an semantic-equivalent while statement, and vice versa
3-> Change the assignment x++ into x=x+1 or x=x+1 into x+=1
4-> Merge the declaration statements into a single composite declaration statement
5-> Divide the composite declaration statement into separated declaration statements
6-> Switch the two code blocks in the if statement and the corresponding else statement
7-> Change a single if statement into a conditional expression statement.
8-> Change a conditional expression statement into a single if statement
9-> Divide a infix expression into two expressions whose values are stored in temporary variables
10-> Divide a if statement with a compound condition (∧ , ∨, or ¬) into two nested if statements
11-> Switch the places of two adjacent statements in a code block, where the former statement has no shared variable with the latter statement
12-> Replace the if-continue statement in a loop block with if-else statement
13-> Switch the two expressions on both sides of the infix expression whose operator is =
14-> Switch the two string objects on both sides of an equality expression
15-> Divide a pre-fix or post-fix expression into two expressions whose values are stored in temporary variables
16-> Transform the ‘‘Switch-Case’’ statement into the corresponding ‘‘If-Else’’ statement.

Code Snippet: "$MY_UNIQUE_PLACE_HOLDER_1"

The following lines are important and should be included in the final result, but they can still be changed using only the first 5 rules, the rest may be changed using any of the rules or can even be removed if they have no relation to these lines: (Lines are separated via /~/)

"$MY_UNIQUE_PLACE_HOLDER_2"

Put the generated code inside ```C ```. (Do not include comments)
"""  # 2 of the rules were merged for simplicity
}

class CodeQwenPrompter:
    def __init__(self, system_message="You are a helpful assistant.", lock=None) -> None:
        self.messages = None
        self.system_message = system_message
        self.tokenizer = AutoTokenizer.from_pretrained(CODEQWEN_ARCHITECTURE)
        self.lock = lock
        self.prompt = self._prompt
        if lock != None:
            self.selector = 0
            self.prompt=self._multi_model_prompt
            self.model = AutoModelForCausalLM.from_pretrained(CODEQWEN_ARCHITECTURE, torch_dtype="auto", device_map="cuda:0"  # Forces only GPU 0
                                                              , max_memory={0: "8GB", 1: "8GB", 2: "8GB", 3: "8GB"},cache_dir="./model_cache/")
            self.model2 = AutoModelForCausalLM.from_pretrained(CODEQWEN_ARCHITECTURE, torch_dtype="auto", device_map="cuda:0"  # Forces only GPU 0
                                                               , max_memory={0: "8GB", 1: "8GB", 2: "8GB", 3:"8GB"} ,cache_dir="./model_cache/")
            # self.model3 = AutoModelForCausalLM.from_pretrained(CODEQWEN_ARCHITECTURE, torch_dtype="auto", device_map="balanced", max_memory={0: "8GB", 1: "8GB", 2: "8GB", 3: "8GB"} ,cache_dir="./model_cache/")
            # self.model4 = AutoModelForCausalLM.from_pretrained(CODEQWEN_ARCHITECTURE, torch_dtype="auto", device_map="balanced", max_memory={0: "8GB", 1: "8GB", 2: "8GB", 3: "8GB"} ,cache_dir="./model_cache/")
            # self.models = {0:self.model, 1:self.model2, 2: self.model3, 3:self.model4}
            self.models = {0:self.model, 1:self.model2}

        else:
            self.model = AutoModelForCausalLM.from_pretrained(CODEQWEN_ARCHITECTURE, torch_dtype="auto", device_map="cuda:0", cache_dir="./model_cache/" , max_memory={0: "15GB", 1: "15GB"})

    def _multi_model_prompt(self, message, clean=True, show=False):
        model = None
        print(self.selector)
        with self.lock:
            model = self.models[self.selector % len(self.models)]
            self.selector += 1

                # print(t)
        if self.messages == None or clean:
            self.messages = [
                # system message to set the behavior of the assistant
                {"role": "system", "content": self.system_message},
            ]

        self.messages.append({"role": "user", "content": message})

        text = self.tokenizer.apply_chat_template(
            self.messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = self.tokenizer([text], return_tensors="pt").to(device)
        print(self.tokenizer.decode(text[0]))

        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=4096 # ChatGPT defaults to 4096
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        reply = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        self.messages.append({"role": "assistant", "content": reply})

        if show:
            print(reply)

        return reply


    # clean initially, it is always dirty after use
    def _prompt(self, message, clean=True, show=False):
        # print(t)
        if self.messages == None or clean:
            self.messages = [
                # system message to set the behavior of the assistant
                {"role": "system", "content": self.system_message},
            ]

        self.messages.append({"role": "user", "content": message})

        text = self.tokenizer.apply_chat_template(
            self.messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors="pt").to(device)

        # Ensure pad_token_id is set correctly
        # if self.tokenizer.pad_token_id is None:
        #     self.tokenizer.pad_token = self.tokenizer.eos_token  # Set pad token to eos token

        generated_ids = self.model.generate(
            input_ids=model_inputs.input_ids,
            attention_mask=model_inputs.attention_mask,
            pad_token_id=self.tokenizer.pad_token_id,
            max_new_tokens=1024, # ChatGPT defaults to 4096
            do_sample=True,
            num_beams=4
        )
        # generated_ids = [
        #     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        # ]

        reply = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        self.messages.append({"role": "assistant", "content": reply})

        if show:
            print(reply)

        return reply

class PromptUtils:
    @staticmethod
    def extract_code_content(text):
        print(text)
        start_index = text.find('```C') + 4
        print('start_index')
        print(start_index)
        end_index = text.find('```', start_index)
        print('end_index')
        print(end_index)
        if start_index != -1 and end_index != -1:
            print(text[start_index:end_index].strip())
            return text[start_index:end_index].strip()
        else:
            print("Sth is wrong!")
            return None

    @staticmethod
    def get_vardef(vul, vulnerable_lines):
        # Step 1: Extract the arguments string from the function definition
        # Using re.DOTALL to allow the dot to match newlines
        args_match = re.search(r'\((.*?)\)', vul, re.DOTALL)
        if args_match:
            args_str = args_match.group(1)
        else:
            return ""

        # Step 2: Split the arguments string into individual arguments
        # This uses a regular expression to split on commas not inside parentheses
        full_args = [arg.strip()
                     for arg in re.split(r',\s*(?![^()]*\))', args_str)]

        # Step 3: Extract argument names, ignoring types
        args = []
        for arg in full_args:
            # Handle cases with pointers and references
            arg_name = arg.split()[-1].replace('*', '').replace('&', '')
            args.append(arg_name)

        # Step 4: Check which arguments are used in vulnerable_lines
        used_args = []
        used_full_args = []
        for i, arg in enumerate(args):
            if re.search(r'\b' + re.escape(arg) + r'\b', vulnerable_lines):
                used_args.append(arg)
                used_full_args.append(full_args[i])

        # Step 5: Format and print the results
        return f"(Don't Forget to add these variables to function's input arguments: {', '.join(full_arg for arg, full_arg in zip(used_args, used_full_args))})"

    @staticmethod
    def get_full_prompt_without_examples_and_vardef_guide(vul, clean, vulnerable_lines):
        vardef = PromptUtils.get_vardef(vul, vulnerable_lines)
        return templates["without_example_vardef"].replace(place_holders[1], vul).replace(place_holders[2], clean).replace(place_holders[3], vulnerable_lines).replace(place_holders[4], vardef)

    @staticmethod
    def get_full_prompt_with_examples_and_vardef_guide(fixed, vul, clean, vulnerable_lines):
        vardef = PromptUtils.get_vardef(vul, vulnerable_lines)
        return templates["with_example_vardef"].replace(place_holders[1], fixed).replace(place_holders[2], vul).replace(place_holders[3], clean).replace(place_holders[4], vulnerable_lines).replace(place_holders[5], vardef)

    @staticmethod
    def get_full_prompt(snippet1, snippet2, vulnerable_lines):
        return templates["extra"].replace(place_holders[1], snippet1).replace(place_holders[2], snippet2).replace(place_holders[3], vulnerable_lines)

    @staticmethod
    def get_ext_wl_prompt(clean, vul, vulnerable_lines):
        return templates["ext_wl"].replace(place_holders[1], clean).replace(place_holders[2], vul).replace(place_holders[3], vulnerable_lines)

    @staticmethod
    def get_mutation_prompt(vul, vulnerable_lines):
        return templates["mutation"].replace(place_holders[1], vul).replace(place_holders[2], vulnerable_lines)

    @staticmethod
    def get_ext_prompt(clean, vul):
        return templates["ext"].replace(place_holders[1], clean).replace(place_holders[2], vul)


    @staticmethod
    def get_prompt(snippet1, snippet2):
        return templates["base"].replace(place_holders[1], snippet1).replace(place_holders[2], snippet2)


class ConcurrencyUtils:

    @staticmethod
    def wait_for_all_futures(futures):
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
            except Exception as e:
                print(f"something's up {e}")

    @staticmethod
    def concurrent_chatgpt_call(prompt_creator_from_searchItem: Callable[[SearchItem], str],
                                index_retreiver: Callable[[SearchItem], str],
                                vul_retriever: Callable[[SearchItem], str],
                                clean_retriever: Callable[[SearchItem], str],
                                vul_lines_retriever: Callable[[SearchItem], str],
                                search_item: SearchItem,
                                indices: List, vuls: List, cleans: List, vul_lines: List, generated: List,
                                lock, llm, retry=2) -> None:
        if retry <= 0:
            return


        try:
            prompt = prompt_creator_from_searchItem(search_item)
            # print('prompt')
            # print(prompt)
            code = PromptUtils.extract_code_content(llm.prompt(prompt))
            print('code')
            if code == None:
                print(f"Empty Code response - retry at {retry}")
                ConcurrencyUtils.concurrent_chatgpt_call(prompt_creator_from_searchItem,
                                                     index_retreiver,
                                                     vul_retriever,
                                                     clean_retriever,
                                                     vul_lines_retriever,
                                                     search_item,
                                                     indices, vuls, cleans, vul_lines, generated,
                                                     lock, llm, retry=retry - 1)

            with lock:
                generated.append(code)
                indices.append(index_retreiver(search_item))
                vuls.append(vul_retriever(search_item))
                cleans.append(clean_retriever(search_item))
                vul_lines.append(vul_lines_retriever(search_item))

            print("Success!")

        # except (openai.error.APIError, torch.cuda.OutOfMemoryError) as e:
        #     print(f"ex happend {e}")
        #     # print(f"Arguments: {e.args}")
        #     print(f"Exception class: {e.__class__}")
        #     print(f"Cause: {e.__cause__}")
        #     print(f"Context: {e.__context__}")
        #     print(f"API Error - retry at {retry}")
        #     time.sleep(retry + 1) # maybe the server wasn't available so wait
        #     ConcurrencyUtils.concurrent_chatgpt_call(prompt_creator_from_searchItem,
        #                                              index_retreiver,
        #                                              vul_retriever,
        #                                              clean_retriever,
        #                                              vul_lines_retriever,
        #                                              search_item,
        #                                              indices, vuls, cleans, vul_lines, generated,
        #                                              lock, llm, retry=retry - 1)
        except Exception as e:
            print(f"ex happend {e}")
            print(f"Arguments: {e.args}")
            print(f"Exception class: {e.__class__}")
            print(f"Cause: {e.__cause__}")
            print(f"Context: {e.__context__}")

    @staticmethod
    def concurrent_prompter(prompt_creator_from_searchItem: Callable[[SearchItem], str],
                            index_retreiver: Callable[[SearchItem], str],
                            vul_retriever: Callable[[SearchItem], str],
                            clean_retriever: Callable[[SearchItem], str],
                            vul_lines_retriever: Callable[[SearchItem], str],
                            search_results: List[SearchItem], llm,
                            target: int = 2000, max_workers=60, is_all=False, resume_from_dataframe=None):
        vuls = []
        cleans = []
        vul_lines = []
        indices = []
        generated = []
        starting_index = 0

        if resume_from_dataframe is not None:
            print("resuming from dataframe")
            for index, row in resume_from_dataframe.iterrows():
                generated.append(str(row['generated']))
                indices.append(str(row['idx']))
                vuls.append(str(row["vul"]))
                vul_lines.append(str(row["vul_lines"]))
                cleans.append(str(row["clean"]))

            max_indx = -1
            for idx in indices:
                list_indx = next((j for j, result in enumerate(search_results) if result.best == idx), None)
                if list_indx != None and list_indx > max_indx:
                    max_indx = list_indx

            starting_index = max_indx + 1
        print(f"starting with index: {starting_index}")

        lock = threading.Lock()

        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executors:
            futures = [executors.submit(
                ConcurrencyUtils.concurrent_chatgpt_call,
                prompt_creator_from_searchItem,
                index_retreiver, vul_retriever,
                clean_retriever, vul_lines_retriever,
                search_results[i], indices, vuls, cleans,
                vul_lines, generated, lock, llm) for i in range(starting_index,target)]

            ConcurrencyUtils.wait_for_all_futures(futures)
            end = target

            while len(generated) < target:

                if is_all:
                    break

                remaining = target - len(generated)
                new_limit = end + remaining

                print(f"DEBUG: Done is: {len(generated)}")
                print(f"DEBUG: Last i: {new_limit}")
                futures = [executors.submit(ConcurrencyUtils.concurrent_chatgpt_call,
                                            prompt_creator_from_searchItem,
                                            index_retreiver, vul_retriever,
                                            clean_retriever, vul_lines_retriever,
                                            search_results[i], indices, vuls, cleans,
                                            vul_lines, generated, lock, llm)
                           for i in range(end, new_limit)]
                ConcurrencyUtils.wait_for_all_futures(futures)
                end = new_limit

        return indices, vuls, cleans, vul_lines, generated


In [34]:

# from chat import PromptUtils

extract_code_content = PromptUtils.extract_code_content

# from chat import CodeQwenPrompter as code_qwen
# from chat import PromptUtils as pu

ch = CodeQwenPrompter(lock=None) # Selects CodeQwen1.5--7B-Chat


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [35]:
test = '''' 
Here are two code snippets specified below, modify code snippet No.2 in a way that it includes the logic of code No.1:
Code Snippet 1: "static int
ProcPseudoramiXGetScreenSize(ClientPtr client)
{
    REQUEST(xPanoramiXGetScreenSizeReq);
    WindowPtr pWin;
    xPanoramiXGetScreenSizeReply rep;
    register int rc;

    TRACE;

    if (stuff->screen >= pseudoramiXNumScreens)
      return BadMatch;

    REQUEST_SIZE_MATCH(xPanoramiXGetScreenSizeReq);
    rc = dixLookupWindow(&pWin, stuff->window, client, DixGetAttrAccess);
    if (rc != Success)
        return rc;

    rep.type = X_Reply;
    rep.length = 0;
    rep.sequenceNumber = client->sequence;
    /* screen dimensions */
    rep.width = pseudoramiXScreens[stuff->screen].w;
    // was screenInfo.screens[stuff->screen]->width;
    rep.height = pseudoramiXScreens[stuff->screen].h;
    // was screenInfo.screens[stuff->screen]->height;
    rep.window = stuff->window;
    rep.screen = stuff->screen;
    if (client->swapped) {
        swaps(&rep.sequenceNumber);
        swapl(&rep.length);
        swapl(&rep.width);
        swapl(&rep.height);
        swapl(&rep.window);
        swapl(&rep.screen);
    }
    WriteToClient(client, sizeof(xPanoramiXGetScreenSizeReply),&rep);
    return Success;
}"

Code Snippet 2: "static int
SProcPseudoramiXIsActive(ClientPtr client)
{
    REQUEST(xXineramaIsActiveReq);

    TRACE;

    swaps(&stuff->length);
    REQUEST_SIZE_MATCH(xXineramaIsActiveReq);
    return ProcPseudoramiXIsActive(client);
}"

Note that the following lines from the first code snippet have a high priority to be in the final modified code:

Lines separated by /~/ : "{'deleted_lines': ['    REQUEST_SIZE_MATCH(xPanoramiXGetScreenSizeReq);'], 'added_lines': ['    REQUEST_SIZE_MATCH(xPanoramiXGetScreenSizeReq);', '']}"

Put the generated code inside ```C ```. (Do not include comments)


'''
ch.prompt(test)

'system\nYou are a helpful assistant.\nuser\n\' \nHere are two code snippets specified below, modify code snippet No.2 in a way that it includes the logic of code No.1:\nCode Snippet 1: "static int\nProcPseudoramiXGetScreenSize(ClientPtr client)\n{\n    REQUEST(xPanoramiXGetScreenSizeReq);\n    WindowPtr pWin;\n    xPanoramiXGetScreenSizeReply rep;\n    register int rc;\n\n    TRACE;\n\n    if (stuff->screen >= pseudoramiXNumScreens)\n      return BadMatch;\n\n    REQUEST_SIZE_MATCH(xPanoramiXGetScreenSizeReq);\n    rc = dixLookupWindow(&pWin, stuff->window, client, DixGetAttrAccess);\n    if (rc != Success)\n        return rc;\n\n    rep.type = X_Reply;\n    rep.length = 0;\n    rep.sequenceNumber = client->sequence;\n    /* screen dimensions */\n    rep.width = pseudoramiXScreens[stuff->screen].w;\n    // was screenInfo.screens[stuff->screen]->width;\n    rep.height = pseudoramiXScreens[stuff->screen].h;\n    // was screenInfo.screens[stuff->screen]->height;\n    rep.window = stuff->

In [36]:
ch.tokenizer.apply_chat_template( '')

UndefinedError: str object has no element 0

In [ ]:
!pip show transformers

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: c:\programdata\miniconda3\envs\vulscriber-cluster-env\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


<!-- ## Introduciton of RAG -->

In [38]:
# from rag import read_naive_full_header_rag, read_naive_full_bigvul_header_rag, read_naive_full_func_rag, read_naive_code2code_rag, read_naive_code2code_ext_rag, read_random, read_naive_code2code_clustered_rag, read_random_fair

search_results = read_naive_code2code_clustered_rag() # For Injection and Extension Methods - RQ1 and RQ3


In [39]:
search_results[0].max_score, search_results[1].max_score, search_results[2].max_score, search_results[10].max_score, search_results[100].max_score
len(search_results),search_results[0]

(7059,
 SearchItem(searchItemId='src/C/cholmod.c', scoredCodeSnippets=[
   ScoredCodeSnippet(id='mono/metadata/icall.c', body='static gboolean
 ves_icall_System_Array_FastCopy (MonoArray *...', header='static gboolean ves_icall_System_Array_FastCopy (MonoArray* source, int source_idx, MonoArray* dest, int dest_idx, int length)', cluster='0', score=4.092301)
   ScoredCodeSnippet(id='codestream/acsequentialscan.hpp', body='void Init(bool hi) 
       {
         for(int i = 0;i < 18;i++...', header='void Init(bool hi)', cluster='1', score=3.7259467)
   ScoredCodeSnippet(id='src/C/cholmod.c', body='static PyObject* spsolve(PyObject *self, PyObject *args,
    ...', header='static PyObject* spsolve(PyObject* self, PyObject* args, PyObject* kwrd)', cluster='2', score=18.262472)
   ScoredCodeSnippet(id='src/imagew-bmp.c', body='static int find_high_bit(unsigned int x)
 {
 	int i;
 	for(i=31...', header='static int find_high_bit(unsigned int x)', cluster='3', score=4.4920135)
   ScoredCodeSnippe

<!-- Vì MegaVul không có trường metadata riêng cho vulnerable_lines, chúng ta sẽ giả định rằng các dòng code bị xóa (deleted_lines) trong quá trình sửa lỗi (tức là trong diff_line_info) chính là các dòng chứa hoặc liên quan trực tiếp đến lỗ hổng trong phiên bản code trước đó (func_before). -->

In [ ]:
# from rag import flatten_search_items
from operator import attrgetter

def sort(search_results):
    return sorted(search_results, key=attrgetter('max_score'), reverse=True)

print(len(search_results))
search_results = flatten_search_items(search_results) # flattens search results in case the format is (1 clean) -> 5 vuls (for RQ2) instead of 1-1 matching, otherwise it doesn't change anything

search_results = sort(search_results) # sort the results (if clustering is used, this will be invalidated and the clustered sampling algorithm's sorting method will be used)
print(len(search_results))


7059
35167


In [42]:

avg = 0
max_avg = 0
avg_m = 0
for r in search_results:
    if avg_m < r.avg_score:
        avg_m = r.avg_score
    avg += r.avg_score / len(search_results)
    max_avg += r.max_score/len(search_results)

print(avg, max_avg, avg_m)

10.350285964307089 10.350285964307089 89.021286


In [43]:
def clustered_sampling(search_results):
    if search_results[0].scoredCodeSnippets[0].cluster == -1:
        print("Search results are for a non-clustered strategy, skipping clustred_sampling")
        return search_results

    cluster_order = [4,3,1,0,2] # sorted based on size
    cluster_indices = [0,0,0,0,0]

    new_search_results = []

    search_results_per_cluster = {0:[], 1:[], 2:[],3:[],4:[]}

    for sr in search_results:
        search_results_per_cluster[sr.scoredCodeSnippets[0].cluster].append(sr) # putting items of each cluster to it's array

    for i in range(5):
        search_results_per_cluster[i] = sort(search_results_per_cluster[i]) # sort each cluster's results

    for i in range(len(search_results)):
        cluster_number = cluster_order[i%5] # sampling from sorted clusters while starting from the largest cluster
        cluster_index = cluster_indices[cluster_number]

        if cluster_index < len(search_results_per_cluster[cluster_number]):
            new_search_results.append(search_results_per_cluster[cluster_number][cluster_index])
            cluster_indices[cluster_number] += 1

    print("Clusted Sampling was used...")
    return new_search_results

search_results = clustered_sampling(search_results)

Clusted Sampling was used...


In [44]:
search_results[0].max_score, search_results[1].max_score, search_results[2].max_score, search_results[10].max_score, search_results[100].max_score

(79.94688, 41.326096, 66.86675, 74.269684, 56.64057)

In [45]:
# resuming_data_frame = pd.read_json("./generated/"somesetting".jsonl", orient="records", lines=True) # just in case more was needed, but didn't want to start from 0 again

In [ ]:
import numpy as np
# from rag import SearchItem
# from chat import ConcurrencyUtils as cu

def get_index(search_item: SearchItem):
    return str(search_item.best)


def get_vul(search_item: SearchItem):
    best_target_index = search_item.best
    vul_record = vul_df[vul_df['file_path'] == best_target_index]
    return str(vul_record['func_before'].values[0])

def get_fixed_vul(search_item: SearchItem):
    best_target_index = search_item.best
    vul_record = vul_df[vul_df['file_path'] == best_target_index]
    return str(vul_record['func'].values[0])

# def get_vul_lines(search_item: SearchItem):
#     best_target_index = search_item.best
#     vul_record = vul_df[vul_df['file_path'] == best_target_index]
#     flaw_lines = str(vul_record['diff_line_info'].values[0]).strip()
#     if len(flaw_lines) < 5:
#         raise Exception(f"Invalid Vul_lines: {flaw_lines}")
#     return flaw_lines
def get_vul_lines(search_item: SearchItem):
    best_target_index = search_item.best
    vul_record = vul_df[vul_df['file_path'] == best_target_index]

    # Debugging step: Print matched rows
    # print(f"Matching records for {best_target_index}:")
    # print(vul_record)

    # Check if record exists
    if vul_record.empty:
        raise Exception(f"No matching record found for file_path: {best_target_index}")

    # Check if `diff_line_info` exists
    if "diff_line_info" not in vul_record.columns:
        raise Exception(f"Missing `diff_line_info` column in vul_df!")

    # Extract flaw lines
    flaw_lines = vul_record['diff_line_info'].values[0]

    # Handle missing or empty flaw_lines
    if flaw_lines is None or str(flaw_lines).strip() == "":
        raise Exception(f"Missing or empty `diff_line_info` for: {best_target_index}")

    return str(flaw_lines).strip()

# def get_clean_megavul(search_item: SearchItem):
#     with open(f"./../devign_code/{search_item.searchItemId}", 'r') as file:
#         clean_item = file.read()
#     return clean_item

def get_clean_megavul(search_item: SearchItem):
    best_target_index = search_item.best
    return str(clean_df[clean_df['file_path'] == best_target_index]['func'].values[0])

# def get_clean_megavul(search_item: SearchItem):
#     # Search for the function by matching `searchItemId` with the `file_path` key
#     for item in clean_json_data:
#         if item.get("file_path") == search_item.searchItemId:
#             return item.get("func", None)  # Return function content if found

#     return None  # Return None if not found

# Formulator Functions that take a search result and populates a prompt template

def get_injection_strategy_devign(search_item: SearchItem): # the Injection strategy while the clean's come from devign
    clean_item = get_clean_megavul(search_item)
    vul_code = get_vul(search_item)
    flaw_lines = get_vul_lines(search_item)
    return PromptUtils.get_full_prompt(vul_code, clean_item, flaw_lines)


# def get_extension_strategy_devign(search_item: SearchItem):
#     clean_item = get_clean_megavul(search_item)
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return PromptUtils.get_ext_wl_prompt(clean_item, vul_code, flaw_lines)

# def get_mutation_strategy(search_item: SearchItem):
#     vul_code = get_vul(search_item)
#     flaw_lines = get_vul_lines(search_item)
#     return PromptUtils.get_mutation_prompt(vul_code, flaw_lines)


# Generator Phase (can send concurrent requests)
indices, vuls, cleans, vul_lines, generated = ConcurrencyUtils.concurrent_prompter(
    prompt_creator_from_searchItem=get_injection_strategy_devign,
    index_retreiver=get_index,
    vul_retriever=get_vul,
    clean_retriever=get_clean_megavul,
    vul_lines_retriever=get_vul_lines,
    search_results=search_results,
    llm=ch, target=6000, max_workers=8, is_all=False, resume_from_dataframe=None)  # for code Qwen set workers to 1 as there's only one model


starting with index: 0
system
You are a helpful assistant.
user

Here are two code snippets specified below, modify code snippet No.2 in a way that it includes the logic of code No.1:
Code Snippet 1: "int GENERAL_NAME_cmp(GENERAL_NAME *a, GENERAL_NAME *b)
{
    int result = -1;

    if (!a || !b || a->type != b->type)
        return -1;
    switch (a->type) {
    case GEN_X400:
        result = ASN1_TYPE_cmp(a->d.x400Address, b->d.x400Address);
        break;

    case GEN_EDIPARTY:
        result = edipartyname_cmp(a->d.ediPartyName, b->d.ediPartyName);
        break;

    case GEN_OTHERNAME:
        result = OTHERNAME_cmp(a->d.otherName, b->d.otherName);
        break;

    case GEN_EMAIL:
    case GEN_DNS:
    case GEN_URI:
        result = ASN1_STRING_cmp(a->d.ia5, b->d.ia5);
        break;

    case GEN_DIRNAME:
        result = X509_NAME_cmp(a->d.dirn, b->d.dirn);
        break;

    case GEN_IPADD:
        result = ASN1_OCTET_STRING_cmp(a->d.ip, b->d.ip);
        break;

    cas

In [ ]:
# save the results of the Generator
df_result = pd.DataFrame({'vul': vuls, 'vul_lines': vul_lines, 'clean': cleans, 'generated': generated, 'idx':indices})
df_result.to_json(f"./generated/6k-devign-injection-clusterfair.jsonl", orient="records", lines=True)